In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import uuid
import shutil
from PIL import Image
import zipfile
import requests

In [2]:
dataset_file=requests.get("https://github.com/ShaharyarAhmed/deep-learning-literature-review/raw/refs/heads/main/Chess_pieces.zip").content
with open("Chess_pieces.zip", "wb") as f:
    f.write(dataset_file)

with zipfile.ZipFile("Chess_pieces.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [3]:
def change_to_black_and_white(image_path, output_path):

    img_cv = cv2.imread(image_path)
    hls = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HLS)
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([100, 255, 255])

    hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    hls_modified = hls.copy()
    hls_modified[mask_green > 0, 2] = 0
    hls_modified[mask_green > 0, 0] = 0
    result_bgr = cv2.cvtColor(hls_modified, cv2.COLOR_HLS2BGR)
    result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)
    Image.fromarray(result_rgb).save(output_path)


In [4]:
folders=['train','valid','test']
for folder in folders:
    image_folder = f'Chess_pieces/{folder}/images'
    txt_folder = f'Chess_pieces/{folder}/labels'
    files_name = [f for f in os.listdir(image_folder)]

    for file_name in files_name:
        original_image_path = os.path.join(image_folder, file_name)
        original_txt_path = os.path.join(txt_folder, file_name.replace('.jpg', '.txt'))
        if not os.path.exists(original_txt_path):
            continue
        new_name = str(uuid.uuid4())
        new_image_path = os.path.join(image_folder, new_name+'.jpg')
        new_txt_path = os.path.join(txt_folder, new_name+'.txt')

        new_name_bnw = os.path.join(image_folder, new_name+'_bnw.jpg')
        change_to_black_and_white(original_image_path, new_name_bnw)
        new_name_bnw_txt = os.path.join(txt_folder, new_name+'_bnw.txt')
    
        os.rename(original_image_path, new_image_path)
        os.rename(original_txt_path, new_txt_path)
        shutil.copyfile(new_txt_path, new_name_bnw_txt)
        

    print(f'Folder {folder} done')


Folder train done
Folder valid done
Folder test done
